# Twitter Scraper

In [1]:
# Import packages
import tweepy
import pandas as pd
import datetime as dt

In [2]:
# Access Twitter API
CONSUMER_KEY = 'PqIOkmsFKDc8lIoSaDGf0Kcr0'
CONSUMER_SECRET = '50kaIGjkmKSSFTKoFtte8j6vbR0CX7KPUDqB4oRhpW0k7zPuY4'
ACCESS_TOKEN = '1347086178365829123-nDPxqMJfjuf9qgVqOUQ8A8LsfZw7pD'
ACCESS_TOKEN_SECRET = 'gT0tbfA9Ca97GbyAwODU8W453sSxLPpNwv800PFpvnXYo'
# Bearer Token = 'AAAAAAAAAAAAAAAAAAAAAPA%2BMwEAAAAAYYSYRh%2B8n9rbJaxva3Y%2FWqtOidU%3D5wukphfJfVhYaxG6HYIPF7nbEI0f5ShwXgs3U8ccIhTC16rSxL'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

### Scrape by Search Term

In [3]:
# Function to scrape twitter
def scrape_twitter(search_word, num_tweets):
    
  # Scrape tweets
  tweets = tweepy.Cursor(api.search,
                          q=search_word + " -filter:retweets",
                          wait_on_rate_limit = True,
                          wait_on_rate_limit_notify = True,
                          lang="en",
                          count=num_tweets,
                          since=dt.date.today()).items()

  # Post data dictionary
  post_dict = {
      
    "timestamp_scrape" : [],  # timestamp of the scrape
    "date_scrape" : [],       # date of the scrape
    "timestamp_tweet" : [],   # timestamp of the tweet
    "search_word": [],        # requested query 
    "tweet_id" : [],          # id of tweet
    "user_name": [],          # name of user who tweeted
    "user_location": [],      # location of user 
    "text" : [],              # text of the tweet
    "retweet_count": [],      # number of retweets
    "favorite_count": []      # number of likes
  }

  # Collect a list of tweets
  for tweet in tweets:
    post_dict['timestamp_scrape'].append(pd.to_datetime("now"))
    post_dict['date_scrape'].append(dt.date.today())
    post_dict["timestamp_tweet"].append(tweet.created_at)
    post_dict["search_word"].append(search_word)
    post_dict["tweet_id"].append(tweet.id)    
    post_dict["user_name"].append(tweet.user.screen_name)       
    post_dict["user_location"].append(tweet.user.location)   
    post_dict["text"].append(tweet.text)
    post_dict["retweet_count"].append(tweet.retweet_count)
    post_dict["favorite_count"].append(tweet.favorite_count)

  twitter_data = pd.DataFrame(post_dict)
  return twitter_data

In [4]:
# Sample query
search_word = 'bitcoin'
date_today = dt.date.today()
num_tweets = 50

test_df = scrape_twitter(search_word, num_tweets)
print(test_df.shape)

AttributeError: 'API' object has no attribute 'search'

### Scrape by User Account

In [5]:
def scrape_user_tweets(screen_name_list, num_tweets):
    
  # Post data dictionary
  post_dict = {
    "timestamp_scrape" : [],  # timestamp of the scrape
    "date_scrape" : [],       # date of the scrape       
    "timestamp_tweet" : [],   # timestamp of the tweet
    "tweet_id" : [],          # id of tweet
    "user_name": [],          # name of user who tweeted
    "user_location": [],      # location of user 
    "text" : [],              # text of the tweet
    "retweet_count": [],      # number of retweets
    "favorite_count": [],     # number of likes
    "account_followers": []   # number of account followers
  }
  
  for user in screen_name_list:
      
    for page_num in range(5):
      
      # Scrape tweets
      user_tweets = api.user_timeline(screen_name=user,
                                      page=page_num+1, 
                                      include_rts = False,
                                      wait_on_rate_limit = True, 
                                      wait_on_rate_limit_notify = True)

      # Collect a list of tweets
      for tweet in user_tweets:
        post_dict['timestamp_scrape'].append(pd.to_datetime("now"))
        post_dict['date_scrape'].append(dt.date.today())
        post_dict["timestamp_tweet"].append(tweet.created_at)
        post_dict["tweet_id"].append(tweet.id)    
        post_dict["user_name"].append(tweet.user.screen_name)       
        post_dict["user_location"].append(tweet.user.location)   
        post_dict["text"].append(tweet.text)
        post_dict["retweet_count"].append(tweet.retweet_count)
        post_dict["favorite_count"].append(tweet.favorite_count)

        account = api.get_user(user)
        post_dict["account_followers"].append(account.followers_count)

    twitter_data = pd.DataFrame(post_dict)
    
    twitter_data = twitter_data.groupby('user_name').apply(lambda x: x.sort_values(['timestamp_tweet'], ascending=False).head(num_tweets))
    twitter_data.reset_index(drop=True, inplace=True)

  return twitter_data

In [6]:
# Sample query
screen_name_list = ['@Bitcoin', '@ethereum', '@LTCFoundation', '@Algorand', '@Cardano', '@Tether_to', '@Ripple', '@cosmos', '@Polkadot', '@dogecoin']
num_tweets = 10

test_df = scrape_user_tweets(screen_name_list, num_tweets)
print(test_df.shape)
print(test_df.user_name.value_counts())
test_df.head(50)

Unexpected parameter: page
Unexpected parameter: wait_on_rate_limit
Unexpected parameter: wait_on_rate_limit_notify


TypeError: get_user() takes 1 positional argument but 2 were given

In [ ]:
test_df.user_name.unique()

array(['Bitcoin', 'ethereum', 'litecoin', 'Algorand', 'Cardano',
       'Tether_to', 'Ripple', 'cosmos', 'Polkadot', 'dogecoin'],
      dtype=object)